<a href="https://colab.research.google.com/github/henryjhu/Anomaly-Detection-in-Wire-Activities/blob/main/DSC_680_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DSC-680-Z1 Research Practicum** <BR> Machine Learning

## **Project Description**
A global bank sought to find new and innovative means for detecting and preventing fraud in their wire transactions. Their goal is through machine learning to arrive at scenario detection rules which could be customized with parameters specific for each cohort of customers. The immediate challenge was that the data provided had no labels. Therefore, unsupervised machine learning techniques were first employed for this research project. After the labels have been identified, supervised machine learning techniques were then employed with selected tuning parameters to increase both the sensitivity and specificity percentages of both classes.

<b>Purpose:</b><br>
Carry out supervised machine learnings with the sample data.<br>
<b>Universtiy Name:</b> Utica College <br>
<b>Course Name:</b> DSC-680-Z1 Research Practicum <br>
<b>Student Name:</b> Henry J. Hu <br>
<b>Program Director Name:</b> Dr. McCarthy, Michael <br>
<b>Runtime Environment:</b> Google Colab<br>
<b>Programming Language:</b> Python <br>
<b>Sample Data Frame:</b>
Six random samples of labeled international wires belonging to 139 customers from 3 continents for the entire year of 2020.<br>
<b> Last Update:</b> August 7th, 2021

## **Installing Packages Into Google Colab**

In [1]:
!pip install -U imbalanced-learn

## **Mounting Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


## **Importing Libraries**

In [3]:
# Importing libraries
from imblearn.over_sampling import SMOTE # https://github.com/vsmolyakov/experiments_with_python/blob/master/chp01/imbalanced_data.ipynb
import io
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import seaborn as sns
import sklearn
import traceback
import time
import pytz
import xgboost as xgb
from datetime import datetime
from sklearn import metrics
from scipy.stats import rankdata
from numpy import quantile, where, random
from random import sample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing
from sklearn.datasets import make_blobs
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_validate 
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, 
                              ExtraTreesClassifier, IsolationForest, VotingClassifier, StackingClassifier)

## **Importing Data into Google Colab**

In [4]:
# Importing data
NN_103_output_df = pd.read_csv("gdrive/MyDrive/DSC-380/NN_103_score_df.cvs")
NN_202_output_df = pd.read_csv("gdrive/MyDrive/DSC-380/NN_202_score_df.cvs")
EU_103_output_df = pd.read_csv("gdrive/MyDrive/DSC-380/EU_103_score_df.cvs")
EU_202_output_df = pd.read_csv("gdrive/MyDrive/DSC-380/EU_202_score_df.cvs")
AS_103_output_df = pd.read_csv("gdrive/MyDrive/DSC-380/AS_103_score_df.cvs")
AS_202_output_df = pd.read_csv("gdrive/MyDrive/DSC-380/AS_202_score_df.cvs")

## **Re-Sampling**

In [5]:
NN_103_output_df = NN_103_output_df.sample(n=100000, random_state=42)
NN_202_output_df = NN_202_output_df.sample(n=100000, random_state=42)
EU_103_output_df = EU_103_output_df.sample(n=100000, random_state=42)
EU_202_output_df = EU_202_output_df.sample(n=100000, random_state=42)
AS_103_output_df = AS_103_output_df.sample(n=100000, random_state=42)
AS_202_output_df = AS_202_output_df.sample(n=100000, random_state=42)

## **Preparation of Training and Test Data Sets**

In [6]:
NN_103_output_df_x_train, NN_103_output_df_x_test, NN_103_output_df_y_train, NN_103_output_df_y_test = train_test_split(NN_103_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],NN_103_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

In [7]:
NN_202_output_df_x_train, NN_202_output_df_x_test, NN_202_output_df_y_train, NN_202_output_df_y_test = train_test_split(NN_202_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],NN_202_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

In [8]:
EU_103_output_df_x_train, EU_103_output_df_x_test, EU_103_output_df_y_train, EU_103_output_df_y_test = train_test_split(EU_103_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],EU_103_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

In [9]:
EU_202_output_df_x_train, EU_202_output_df_x_test, EU_202_output_df_y_train, EU_202_output_df_y_test = train_test_split(EU_202_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],EU_202_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

In [10]:
AS_103_output_df_x_train, AS_103_output_df_x_test, AS_103_output_df_y_train, AS_103_output_df_y_test = train_test_split(AS_103_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],AS_103_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

In [11]:
AS_202_output_df_x_train, AS_202_output_df_x_test, AS_202_output_df_y_train, AS_202_output_df_y_test = train_test_split(AS_202_output_df[['TRXN_MONTH','TRANSACTION_AMOUNT']],AS_202_output_df[['FRAUD_LABEL']],train_size=0.7, test_size=0.3, shuffle=True, random_state=42)

## **Over Sampling the Minority Class Using SMOTE**

In [12]:
# Initilize SMOTE
sm = SMOTE(random_state=42, n_jobs=-1)

# Apply SMOTE to North America
NN_103_sm_output_df_x_train, NN_103_sm_output_df_y_train = sm.fit_resample(NN_103_output_df_x_train, NN_103_output_df_y_train)
NN_103_sm_output_df_x_test, NN_103_sm_output_df_y_test = sm.fit_resample(NN_103_output_df_x_test, NN_103_output_df_y_test)
NN_202_sm_output_df_x_train, NN_202_sm_output_df_y_train = sm.fit_resample(NN_202_output_df_x_train, NN_202_output_df_y_train)
NN_202_sm_output_df_x_test, NN_202_sm_output_df_y_test = sm.fit_resample(NN_202_output_df_x_test, NN_202_output_df_y_test)

#Apply SMOTE to Europe
EU_103_sm_output_df_x_train, EU_103_sm_output_df_y_train = sm.fit_resample(EU_103_output_df_x_train, EU_103_output_df_y_train)
EU_103_sm_output_df_x_test, EU_103_sm_output_df_y_test = sm.fit_resample(EU_103_output_df_x_test, EU_103_output_df_y_test)
EU_202_sm_output_df_x_train, EU_202_sm_output_df_y_train = sm.fit_resample(EU_202_output_df_x_train, EU_202_output_df_y_train)
EU_202_sm_output_df_x_test, EU_202_sm_output_df_y_test = sm.fit_resample(EU_202_output_df_x_test, EU_202_output_df_y_test)

#Apply SMOTE to Asia
AS_103_sm_output_df_x_train, AS_103_sm_output_df_y_train = sm.fit_resample(AS_103_output_df_x_train, AS_103_output_df_y_train)
AS_103_sm_output_df_x_test, AS_103_sm_output_df_y_test = sm.fit_resample(AS_103_output_df_x_test, AS_103_output_df_y_test)
AS_202_sm_output_df_x_train, AS_202_sm_output_df_y_train = sm.fit_resample(AS_202_output_df_x_train, AS_202_output_df_y_train)
AS_202_sm_output_df_x_test, AS_202_sm_output_df_y_test = sm.fit_resample(AS_202_output_df_x_test, AS_202_output_df_y_test)

# unique, count = np.unique(NN_103_sm_output_df_y_train, return_counts=True)
# y_train_ct = {k:v for (k,v) in zip(unique, count)}
# y_train_ct
# NN_103_sm_output_df_x_train

## **Cross Validation Function**

In [13]:
##############################################################################################
#
# Purpose: 
# Through cross validation, performs model fitting, training and prediction for 
# Random Forest and Artificial Neuro Networks 
#
##############################################################################################

In [14]:
def cross_val_train (clf_n, X_train, Y_train):

  # Initialize the log variable
  class log:
    def_tz = pytz.timezone('America/New_York')
    def info(text):        
        print(f'{datetime.now(log.def_tz).replace(microsecond=0)} : {text}');

  # Scale and center the data around the mean of 0
  scaling=StandardScaler()
  X_train=scaling.fit_transform(X_train)

  # Random Seed
  random.seed(42)

  clf1=MLPClassifier()
  clf2=RandomForestClassifier()

  # Best tunning hyper parameters have been determined with two random iterations
  # Data frames used: NN_202_output_df_x_train, NN_202_output_df_y_train
  # {'warm_start': True, 'solver': 'adam', 'random_state': 42, 'max_iter': 300, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (4, 8), 'alpha': 0.05, 'activation': 'logistic'}
  clf1_parameter_space = {
      'warm_start': [True],
      'max_iter': [500],
      'hidden_layer_sizes': [(8,16)],
      'activation': ['logistic'],
      'solver': ['lbfgs'], # Have to set learning_rate_init if solver is either ’sgd’ or ‘adam’ 
      'alpha': [0.05,0.1,1.5,2.0,2.5], # Have to set alpha correctly or else all same label values in output 
      'learning_rate': ['adaptive'],
      'random_state': [42]
  }
  # Best tunning hyper parameters have been determined with two random iterations
  # Data frames used: NN_202_output_df_x_train, NN_202_output_df_y_train
  clf2_parameter_space = {
      'max_depth': [40],
      'n_estimators': [100],
      'min_samples_split': [5],
      'random_state': [42],
      'n_jobs': [-1]
  }

  if clf_n == 1:
    model = clf1
    model_name = 'Artificial Neuro Networks'
    clf_parameter_space = clf1_parameter_space
  else: 
    model = clf2
    model_name = 'Random Forest'
    clf_parameter_space = clf2_parameter_space

  clf_cross = RandomizedSearchCV(model, clf_parameter_space, n_jobs=-1, cv=10, n_iter=1, verbose=True, refit=True) # Stratified cv fold cross validation
  clf_cross.fit(X_train, Y_train) # X is training sample and y is the corresponding labels
  log.info(f'Best parameters used for cross validation of {model_name}')
  log.info(clf_cross.best_params_)

  return clf_cross

## **Stacking - Level 1**

### **Gathering Predictions for Test Data Set X**

### ***Artificial Neuro Networks Predictions***

#### ***North America***

In [15]:
NN_103_sm_output_ann_y_pred_test = cross_val_train(1,NN_103_sm_output_df_x_train, NN_103_sm_output_df_y_train).predict(NN_103_sm_output_df_x_test)
NN_103_sm_output_ann_y_pred_train = cross_val_train(1,NN_103_sm_output_df_x_train, NN_103_sm_output_df_y_train).predict(NN_103_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


2021-08-09 03:30:55-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-09 03:30:55-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 500, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (8, 16), 'alpha': 0.1, 'activation': 'logistic'}
Fitting 10 folds for each of 1 candidates, totalling 10 fits


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


2021-08-09 03:36:32-04:00 : Best parameters used for cross validation of Artificial Neuro Networks
2021-08-09 03:36:32-04:00 : {'warm_start': True, 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 500, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (8, 16), 'alpha': 0.1, 'activation': 'logistic'}


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
NN_202_sm_output_ann_y_pred_test = cross_val_train(1,NN_202_sm_output_df_x_train, NN_202_sm_output_df_y_train).predict(NN_202_sm_output_df_x_test)
NN_202_sm_output_ann_y_pred_train = cross_val_train(1,NN_202_sm_output_df_x_train, NN_202_sm_output_df_y_train).predict(NN_202_sm_output_df_x_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


#### ***Europe***

In [ ]:
EU_103_sm_output_ann_y_pred_test = cross_val_train(1, EU_103_sm_output_df_x_train, EU_103_sm_output_df_y_train).predict(EU_103_sm_output_df_x_test)
EU_103_sm_output_ann_y_pred_train = cross_val_train(1, EU_103_sm_output_df_x_train, EU_103_sm_output_df_y_train).predict(EU_103_sm_output_df_x_train)

In [ ]:
EU_202_sm_output_ann_y_pred_test = cross_val_train(1, EU_202_sm_output_df_x_train, EU_202_sm_output_df_y_train).predict(EU_202_sm_output_df_x_test)
EU_202_sm_output_ann_y_pred_train = cross_val_train(1, EU_202_sm_output_df_x_train, EU_202_sm_output_df_y_train).predict(EU_202_sm_output_df_x_train)

#### ***Asia***

In [ ]:
AS_103_sm_output_ann_y_pred_test = cross_val_train(1, AS_103_sm_output_df_x_train, AS_103_sm_output_df_y_train).predict(AS_103_sm_output_df_x_test)
AS_103_sm_output_ann_y_pred_train = cross_val_train(1, AS_103_sm_output_df_x_train, AS_103_sm_output_df_y_train).predict(AS_103_sm_output_df_x_train)

In [ ]:
AS_202_sm_output_ann_y_pred_test = cross_val_train(1, AS_202_sm_output_df_x_train, AS_202_sm_output_df_y_train).predict(AS_202_sm_output_df_x_test)
AS_202_sm_output_ann_y_pred_train = cross_val_train(1, AS_202_sm_output_df_x_train, AS_202_sm_output_df_y_train).predict(AS_202_sm_output_df_x_train)

### ***Random Forest Predictions***

#### ***North America***

In [ ]:
NN_103_sm_output_rf_y_pred_test = cross_val_train(2,NN_103_sm_output_df_x_train, NN_103_sm_output_df_y_train).predict(NN_103_sm_output_df_x_test)
NN_103_sm_output_rf_y_pred_train = cross_val_train(2,NN_103_sm_output_df_x_train, NN_103_sm_output_df_y_train).predict(NN_103_sm_output_df_x_train)

In [ ]:
NN_202_sm_output_rf_y_pred_test = cross_val_train(2,NN_202_sm_output_df_x_train, NN_202_sm_output_df_y_train).predict(NN_202_sm_output_df_x_test)
NN_202_sm_output_rf_y_pred_train = cross_val_train(2,NN_202_sm_output_df_x_train, NN_202_sm_output_df_y_train).predict(NN_202_sm_output_df_x_train)

#### ***Europe***

In [ ]:
EU_103_sm_output_rf_y_pred_test = cross_val_train(2, EU_103_sm_output_df_x_train, EU_103_sm_output_df_y_train).predict(EU_103_sm_output_df_x_test)
EU_103_sm_output_rf_y_pred_train = cross_val_train(2, EU_103_sm_output_df_x_train, EU_103_sm_output_df_y_train).predict(EU_103_sm_output_df_x_train)

In [ ]:
EU_202_sm_output_rf_y_pred_test = cross_val_train(2, EU_202_sm_output_df_x_train, EU_202_sm_output_df_y_train).predict(EU_202_sm_output_df_x_test)
EU_202_sm_output_rf_y_pred_train = cross_val_train(2, EU_202_sm_output_df_x_train, EU_202_sm_output_df_y_train).predict(EU_202_sm_output_df_x_train)

#### ***Asia***

In [ ]:
AS_103_sm_output_rf_y_pred_test = cross_val_train(2, AS_103_sm_output_df_x_train, AS_103_sm_output_df_y_train).predict(AS_103_sm_output_df_x_test)
AS_103_sm_output_rf_y_pred_train = cross_val_train(2, AS_103_sm_output_df_x_train, AS_103_sm_output_df_y_train).predict(AS_103_sm_output_df_x_train)

In [ ]:
AS_202_sm_output_rf_y_pred_test = cross_val_train(2, AS_202_sm_output_df_x_train, AS_202_sm_output_df_y_train).predict(AS_202_sm_output_df_x_test)
AS_202_sm_output_rf_y_pred_train = cross_val_train(2, AS_202_sm_output_df_x_train, AS_202_sm_output_df_y_train).predict(AS_202_sm_output_df_x_train)

## **Stacking - Level 2**

##### **Stacking Algorithm = Decision Tree**

##### **Weights of 2x applied to Level 1 Random Forest's predictions before feeding them into Level 2 Decesion Tree as inputs**

##### **Level 2 X Training Data = Combine of Level 1 ANN and Random Forest predicitons from the "train" data**

##### **Level 2 X Test Data = Combine of Level 1 ANN and Random Forest predicitons from the "test" data**

#### ***North America***

In [ ]:
NN_103_sm_output_ensemble_x_train = np.column_stack((NN_103_sm_output_ann_y_pred_train, NN_103_sm_output_rf_y_pred_train*2))
NN_103_sm_output_ensemble_x_train_df = pd.DataFrame(NN_103_sm_output_ensemble_x_train, columns = ['ANN','RF'])
NN_103_sm_output_ensemble_x_test = np.column_stack((NN_103_sm_output_ann_y_pred_test, NN_103_sm_output_rf_y_pred_test*2))
NN_103_sm_output_ensemble_x_test_df = pd.DataFrame(NN_103_sm_output_ensemble_x_test, columns = ['ANN','RF'])
NN_103_sm_output_final_pred = xgb.XGBClassifier(objective= 'binary:logistic').fit(NN_103_sm_output_ensemble_x_train_df, NN_103_sm_output_df_y_train).predict(NN_103_sm_output_ensemble_x_test_df)

In [ ]:
NN_202_sm_output_ensemble_x_train = np.column_stack((NN_202_sm_output_ann_y_pred_train, NN_202_sm_output_rf_y_pred_train*2))
NN_202_sm_output_ensemble_x_train_df = pd.DataFrame(NN_202_sm_output_ensemble_x_train, columns = ['ANN','RF'])
NN_202_sm_output_ensemble_x_test = np.column_stack((NN_202_sm_output_ann_y_pred_test, NN_202_sm_output_rf_y_pred_test*2))
NN_202_sm_output_ensemble_x_test_df = pd.DataFrame(NN_202_sm_output_ensemble_x_test, columns = ['ANN','RF'])
NN_202_sm_output_final_pred = xgb.XGBClassifier(objective= 'binary:logistic').fit(NN_202_sm_output_ensemble_x_train_df, NN_202_sm_output_df_y_train).predict(NN_202_sm_output_ensemble_x_test_df)

#### ***Europe***

In [ ]:
EU_103_sm_output_ensemble_x_train = np.column_stack((EU_103_sm_output_ann_y_pred_train, EU_103_sm_output_rf_y_pred_train*2))
EU_103_sm_output_ensemble_x_train_df = pd.DataFrame(EU_103_sm_output_ensemble_x_train, columns = ['ANN','RF'])
EU_103_sm_output_ensemble_x_test = np.column_stack((EU_103_sm_output_ann_y_pred_test, EU_103_sm_output_rf_y_pred_test*2))
EU_103_sm_output_ensemble_x_test_df = pd.DataFrame(EU_103_sm_output_ensemble_x_test, columns = ['ANN','RF'])
EU_103_sm_output_final_pred = DecisionTreeClassifier(random_state=42).fit(EU_103_sm_output_ensemble_x_train_df, EU_103_sm_output_df_y_train).predict(EU_103_sm_output_ensemble_x_test_df)

In [ ]:
EU_202_sm_output_ensemble_x_train = np.column_stack((EU_202_sm_output_ann_y_pred_train, EU_202_sm_output_rf_y_pred_train*2))
EU_202_sm_output_ensemble_x_train_df = pd.DataFrame(EU_202_sm_output_ensemble_x_train, columns = ['ANN','RF'])
EU_202_sm_output_ensemble_x_test = np.column_stack((EU_202_sm_output_ann_y_pred_test, EU_202_sm_output_rf_y_pred_test*2))
EU_202_sm_output_ensemble_x_test_df = pd.DataFrame(EU_202_sm_output_ensemble_x_test, columns = ['ANN','RF'])
EU_202_sm_output_final_pred = DecisionTreeClassifier(random_state=42).fit(EU_202_sm_output_ensemble_x_train_df, EU_202_sm_output_df_y_train).predict(EU_202_sm_output_ensemble_x_test_df)

#### ***Asia***

In [ ]:
AS_103_sm_output_ensemble_x_train = np.column_stack((AS_103_sm_output_ann_y_pred_train, AS_103_sm_output_rf_y_pred_train*2))
AS_103_sm_output_ensemble_x_train_df = pd.DataFrame(AS_103_sm_output_ensemble_x_train, columns = ['ANN','RF'])
AS_103_sm_output_ensemble_x_test = np.column_stack((AS_103_sm_output_ann_y_pred_test, AS_103_sm_output_rf_y_pred_test*2))
AS_103_sm_output_ensemble_x_test_df = pd.DataFrame(AS_103_sm_output_ensemble_x_test, columns = ['ANN','RF'])
AS_103_sm_output_final_pred = DecisionTreeClassifier(random_state=42).fit(AS_103_sm_output_ensemble_x_train_df, AS_103_sm_output_df_y_train).predict(AS_103_sm_output_ensemble_x_test_df)

In [ ]:
AS_202_sm_output_ensemble_x_train = np.column_stack((AS_202_sm_output_ann_y_pred_train, AS_202_sm_output_rf_y_pred_train*2))
AS_202_sm_output_ensemble_x_train_df = pd.DataFrame(AS_202_sm_output_ensemble_x_train, columns = ['ANN','RF'])
AS_202_sm_output_ensemble_x_test = np.column_stack((AS_202_sm_output_ann_y_pred_test, AS_202_sm_output_rf_y_pred_test*2))
AS_202_sm_output_ensemble_x_test_df = pd.DataFrame(AS_202_sm_output_ensemble_x_test, columns = ['ANN','RF'])
AS_202_sm_output_final_pred = DecisionTreeClassifier(random_state=42).fit(AS_202_sm_output_ensemble_x_train_df, AS_202_sm_output_df_y_train).predict(AS_202_sm_output_ensemble_x_test_df)

## **Confusion Matricies of Accuracy Statistics**

#### ***North America***

In [ ]:
NN_103_output_df_y_train.shape

In [ ]:
NN_103_output_df_y_train[NN_103_output_df_y_train['FRAUD_LABEL']>=1].shape 

In [ ]:
NN_103_sm_output_df_y_train.shape # original y train

In [ ]:
NN_103_sm_output_df_y_train[NN_103_sm_output_df_y_train['FRAUD_LABEL']>=1].shape # original y train prediction filter

In [ ]:
NN_103_sm_output_ann_y_pred_train.shape # ann y train level 1 predictions

In [ ]:
NN_103_sm_output_rf_y_pred_train.shape # random forest y train level 1 predictions

In [ ]:
NN_103_sm_output_ann_y_pred_train[NN_103_sm_output_ann_y_pred_train>=1].shape # ann y train level 1 prediction filter

In [ ]:
NN_103_sm_output_rf_y_pred_train[NN_103_sm_output_rf_y_pred_train>=1].shape  # random forest y train level 1 prediction filter

In [ ]:
NN_103_sm_output_ensemble_x_train_df.shape # level 2 x train

In [ ]:
NN_103_sm_output_ensemble_x_train_df.shape # level 2 x train 

In [ ]:
NN_103_sm_output_df_y_train.shape # level 2 y train 

In [ ]:
NN_103_sm_output_ensemble_x_train_df[NN_103_sm_output_ensemble_x_train_df['ANN']>=1].shape # level 2 x train ANN filter

In [ ]:
NN_103_sm_output_ensemble_x_train_df[NN_103_sm_output_ensemble_x_train_df['RF']>=1].shape# level 2 x train RF filter

In [ ]:
NN_103_sm_output_ensemble_x_test_df.shape # level 2 x test 

In [ ]:
NN_103_sm_output_ensemble_x_test_df[NN_103_sm_output_ensemble_x_test_df['ANN']>=1].shape # level 2 x test ANN filter

In [ ]:
NN_103_sm_output_ensemble_x_test_df[NN_103_sm_output_ensemble_x_test_df['RF']>=1].shape# level 2 x test RF filter

In [ ]:
NN_103_sm_output_final_pred.shape # level 2 predictions

In [ ]:
NN_103_sm_output_final_pred[NN_103_sm_output_final_pred>=1].shape # level 2 predictions filter

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
# print(classification_report(NN_103_sm_output_df_y_test, NN_103_sm_output_final_pred, target_names=['class 1', 'class 0'])) 
print(confusion_matrix(NN_103_sm_output_df_y_test,NN_103_sm_output_final_pred))
print(classification_report(NN_103_sm_output_df_y_test,NN_103_sm_output_final_pred))

#### ***Europe***

#### ***Asia***